# **Bibliotecas e Funções**

## ***Bibliotecas***

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# Scikit-Learn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, svm, ensemble
from sklearn.neural_network import MLPClassifier

# OpenCV
import cv2

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## ***Acessos Externos***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Overview do Dataset**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Artigos - Nathan/2024 - ICITED/Dados_Tabulados.csv", skiprows=[1,2])

# Imprime algumas informações básicas sobre os dados
print("Numero de linhas: {}".format(df.shape[0]))
print("Numero de colunas: {}".format(df.shape[1]))
print("Dados faltando: {}".format(df.isnull().sum().sum()))

print("\nAmostra:")
display(df.sample(10))

# Cria uma lista com as features
features = list(df.columns)

# print("Features:")
# print("\n".join(["  " + x for x in features]))
print(features)

mpl.rc('font', size=14)

## ***Dados Categóricos***

In [ ]:
# Colunas Não Numéricas
categ_cols = df.select_dtypes(exclude='number').columns

for c in categ_cols:
    display((df[c].value_counts().sort_index()/len(df)*100).round(2).to_frame(c +" (%)").T)

In [ ]:
df = df.drop(['image name', 'image'], axis='columns')

Target = 'category'
display(df.head(10))

## ***Dados Numéricos***

In [ ]:
features = df.select_dtypes(include=np.number).columns
features

# **Preparação dos Dados**

## ***Normalização das Features***

In [ ]:
# Normalização dos Dados
min_max_scaler = MinMaxScaler(feature_range=(0, 1))

df[features] = min_max_scaler.fit_transform(df[features])
df[features].describe()

## ***Target Numérico***

In [ ]:
df[Target] = (df[Target] == "selective-logging").astype(int)
display((df[Target].value_counts()/len(df)*100).round(2).to_frame(Target +" (%)").T)

## ***Split dos Dados***

In [ ]:
x_train, x_rem, y_train, y_rem = train_test_split(df[features], df[Target], train_size=0.8)

x_valid, x_test, y_valid, y_test = train_test_split(x_rem,y_rem, test_size=0.5)

print("Features:")
print("Treinamento:", x_train.shape, "\tValidação:", x_valid.shape, "\tTeste:", x_test.shape)
# print("Treinamento:", x_train.shape, "\tValidação:", x_valid.shape)

print("\nTarget:")
print("Treinamento:", y_train.shape, "\t\tValidação:", y_valid.shape, "\tTeste:", y_test.shape)
# print("Treinamento:", y_train.shape, "\t\tValidação:", y_valid.shape)

# **Aplicação de Redes Neurais**

In [ ]:
x = range(len(features))

## ***KNN (K Nearest Neighbours)***

In [ ]:
# Define uma funcao que recebe o numero k de vizinhos,
def test_knn_ball(k):
  clf = neighbors.KNeighborsClassifier(n_neighbors=k,
                                       algorithm='ball_tree')

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Número de Regiões: {}".format(k))
  print("Acurácia de treino: {:.3}".format(train_acc))
  print("Acurácia de validação: {:.3}".format(val_acc))

def test_knn_tree(k):
  clf = neighbors.KNeighborsClassifier(n_neighbors=k,
                                       algorithm='kd_tree')

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Número de Regiões: {}".format(k))
  print("Acurácia de treino: {:.3}".format(train_acc))
  print("Acurácia de validação: {:.3}".format(val_acc))

def test_knn_brute(k):
  clf = neighbors.KNeighborsClassifier(n_neighbors=k,
                                       algorithm='brute')

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Número de Regiões: {}".format(k))
  print("Acurácia de treino: {:.3}".format(train_acc))
  print("Acurácia de validação: {:.3}".format(val_acc))

In [ ]:
print("Ball:")
test_knn_ball(3)

print("\nTree:")
test_knn_tree(3)

print("\nBrute:")
test_knn_brute(3)

In [ ]:
print("Ball:")
test_knn_ball(5)

print("\nTree:")
test_knn_tree(5)

print("\nBrute:")
test_knn_brute(5)

In [ ]:
print("Ball:")
test_knn_ball(15)

print("\nTree:")
test_knn_tree(15)

print("\nBrute:")
test_knn_brute(15)

In [ ]:
print("Ball:")
test_knn_ball(49)

print("\nTree:")
test_knn_tree(49)

print("\nBrute:")
test_knn_brute(49)

In [ ]:
print("Ball:")
test_knn_ball(99)

print("\nTree:")
test_knn_tree(99)

print("\nBrute:")
test_knn_brute(99)

## ***AdaBoost***

In [ ]:
def test_adaboost(N):
  clf = ensemble.AdaBoostClassifier(n_estimators=N)

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Número de Estimadores: {}".format(N))
  print("Máximas Relevâncias: {:.5}, {:.5}, {:.5}, {:.5}, {:.5}".format(sorted(clf.feature_importances_)[-1],
                                                                        sorted(clf.feature_importances_)[-2],
                                                                        sorted(clf.feature_importances_)[-3],
                                                                        sorted(clf.feature_importances_)[-4],
                                                                        sorted(clf.feature_importances_)[-5]))
  print("Acurácia de treino: {:.3}".format(train_acc))
  print("Acurácia de validação: {:.3}".format(val_acc))

In [ ]:
test_adaboost(3)

In [ ]:
test_adaboost(5)

In [ ]:
test_adaboost(15)

In [ ]:
test_adaboost(49)

In [ ]:
test_adaboost(99)

## ***Random Forest***

In [ ]:
def test_rf(N, max_depth):
  clf = ensemble.RandomForestClassifier(n_estimators=N,
                                        max_depth=max_depth)

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Número de Estimadores: {}".format(N))
  print("Profundidade: {}".format(max_depth))
  print("Máximas Relevâncias: {:.3}, {:.3}, {:.3}, {:.3}, {:.3}".format(sorted(clf.feature_importances_)[-1],
                                                                        sorted(clf.feature_importances_)[-2],
                                                                        sorted(clf.feature_importances_)[-3],
                                                                        sorted(clf.feature_importances_)[-4],
                                                                        sorted(clf.feature_importances_)[-5]))
  print("Acurácia de treino: {:.3}".format(train_acc))
  print("Acurácia de validação: {:.3}".format(val_acc))

In [ ]:
print("Depth None:")
test_rf(3, None)

print("\nDepth 5:")
test_rf(3, 5)

print("\nDepth 10:")
test_rf(3, 10)

In [ ]:
print("Depth None:")
test_rf(5, None)

print("\nDepth 5:")
test_rf(5, 5)

print("\nDepth 10:")
test_rf(5, 10)

In [ ]:
print("Depth None:")
test_rf(15, None)

print("\nDepth 5:")
test_rf(15, 5)

print("\nDepth 10:")
test_rf(15, 10)

In [ ]:
print("Depth None:")
test_rf(49, None)

print("\nDepth 5:")
test_rf(49, 5)

print("\nDepth 10:")
test_rf(49, 10)

In [ ]:
print("Depth None:")
test_rf(99, None)

print("\nDepth 5:")
test_rf(99, 5)

print("\nDepth 10:")
test_rf(99, 10)

## ***SVM (Support Vector Machines)***

In [ ]:
def test_svm_linear(C):
  clf = svm.SVC(C=C,
                kernel='linear',
                gamma='auto')

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Parâmetro Regulador: {}".format(C))
  print("Acuracia de treino: {:.3}".format(train_acc))
  print("Acuracia de validação: {:.3}".format(val_acc))

def test_svm_poly(C):
  clf = svm.SVC(C=C,
                kernel='poly',
                gamma='auto')

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Parâmetro Regulador: {}".format(C))
  print("Acuracia de treino: {:.3}".format(train_acc))
  print("Acuracia de validação: {:.3}".format(val_acc))

def test_svm_rbf(C):
  clf = svm.SVC(C=C,
                kernel='rbf',
                gamma='auto')

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Parâmetro Regulador: {}".format(C))
  print("Acuracia de treino: {:.3}".format(train_acc))
  print("Acuracia de validação: {:.3}".format(val_acc))

def test_svm_sig(C):
  clf = svm.SVC(C=C,
                kernel='sigmoid',
                gamma='auto')

  clf = clf.fit(x_train, y_train)

  train_acc = clf.score(x_train, y_train)
  val_acc = clf.score(x_valid, y_valid)

  print("Parâmetro Regulador: {}".format(C))
  print("Acuracia de treino: {:.3}".format(train_acc))
  print("Acuracia de validação: {:.3}".format(val_acc))

In [ ]:
print("Linear:")
test_svm_linear(.1)

print("\nPolynomial:")
test_svm_poly(.1)

print("\nRBF:")
test_svm_rbf(.1)

print("\nSigmoid:")
test_svm_sig(.1)

In [ ]:
print("Linear:")
test_svm_linear(1)

print("\nPolynomial:")
test_svm_poly(1)

print("\nRBF:")
test_svm_rbf(1)

print("\nSigmoid:")
test_svm_sig(1)

In [ ]:
print("Linear:")
test_svm_linear(5)

print("\nPolynomial:")
test_svm_poly(5)

print("\nRBF:")
test_svm_rbf(5)

print("\nSigmoid:")
test_svm_sig(5)

In [ ]:
print("Linear:")
test_svm_linear(10)

print("\nPolynomial:")
test_svm_poly(10)

print("\nRBF:")
test_svm_rbf(10)

print("\nSigmoid:")
test_svm_sig(10)

## ***MLP (Multilayer Perceptron)***

In [ ]:
def mlp_line(hidden):
    clf = MLPClassifier(hidden_layer_sizes=(hidden),
                        activation='identity',
                        solver='adam',
                        learning_rate='adaptive',
                        max_iter=500,
                        verbose=False)

    clf = clf.fit(x_train, y_train)

    train_acc = clf.score(x_train, y_train)
    val_acc = clf.score(x_valid, y_valid)

    print("Camadas escondidas: {}".format(hidden))
    print("Número de Iterações: {}".format(clf.n_iter_))
    print("Loss Mínimo: {:.5}".format(clf.best_loss_))
    print("Acurácia de treino: {:.3}".format(train_acc))
    print("Acurácia de validação: {:.3}".format(val_acc))

def mlp_relu(hidden):
    clf = MLPClassifier(hidden_layer_sizes=(hidden),
                        activation='relu',
                        solver='adam',
                        learning_rate='adaptive',
                        max_iter=500,
                        verbose=False)

    clf = clf.fit(x_train, y_train)

    train_acc = clf.score(x_train, y_train)
    val_acc = clf.score(x_valid, y_valid)

    print("Camadas escondidas: {}".format(hidden))
    print("Número de Iterações: {}".format(clf.n_iter_))
    print("Loss Mínimo: {:.5}".format(clf.best_loss_))
    print("Acurácia de treino: {:.3}".format(train_acc))
    print("Acurácia de validação: {:.3}".format(val_acc))

def mlp_sigmoid(hidden):
    clf = MLPClassifier(hidden_layer_sizes=(hidden),
                        activation='logistic',
                        solver='adam',
                        learning_rate='adaptive',
                        max_iter=500,
                        verbose=False)

    clf = clf.fit(x_train, y_train)

    train_acc = clf.score(x_train, y_train)
    val_acc = clf.score(x_valid, y_valid)

    print("Camadas escondidas: {}".format(hidden))
    print("Número de Iterações: {}".format(clf.n_iter_))
    print("Loss Mínimo: {:.5}".format(clf.best_loss_))
    print("Acurácia de treino: {:.3}".format(train_acc))
    print("Acurácia de validação: {:.3}".format(val_acc))

def mlp_tanh(hidden):
    clf = MLPClassifier(hidden_layer_sizes=(hidden),
                        activation='tanh',
                        solver='adam',
                        learning_rate='adaptive',
                        max_iter=500,
                        verbose=False)

    clf = clf.fit(x_train, y_train)

    train_acc = clf.score(x_train, y_train)
    val_acc = clf.score(x_valid, y_valid)

    print("Camadas escondidas: {}".format(hidden))
    print("Número de Iterações: {}".format(clf.n_iter_))
    print("Loss Mínimo: {:.5}".format(clf.best_loss_))
    print("Acurácia de treino: {:.3}".format(train_acc))
    print("Acurácia de validação: {:.3}".format(val_acc))

In [ ]:
print("Identidade")
mlp_line((20))

print("\nRelu")
mlp_relu((20))

print("\nSigmoide")
mlp_sigmoid((20))

print("\nTanh")
mlp_tanh((20))

In [ ]:
print("Identidade")
mlp_line((20, 20))

print("\nRelu")
mlp_relu((20, 20))

print("\nSigmoide")
mlp_sigmoid((20, 20))

print("\nTanh")
mlp_tanh((20, 20))

In [ ]:
print("Identidade")
mlp_line((20, 10, 20))

print("\nRelu")
mlp_relu((20, 10, 20))

print("\nSigmoide")
mlp_sigmoid((20, 10, 20))

print("\nTanh")
mlp_tanh((20, 10, 20))

In [ ]:
print("Identidade")
mlp_line((20, 10, 10, 20))

print("\nRelu")
mlp_relu((20, 10, 10, 20))

print("\nSigmoide")
mlp_sigmoid((20, 10, 10, 20))

print("\nTanh")
mlp_tanh((20, 10, 10, 20))

# **Matriz de Confusão**

In [ ]:
classes = (['non-logging', 'selective-loggin'])

In [ ]:
#@title ***KNN***
knn_neighbor = 49

# KNN
knn_clf = neighbors.KNeighborsClassifier(n_neighbors=knn_neighbor)
knn_clf = knn_clf.fit(x_train, y_train)
val_knn = knn_clf.predict(x_test)

cm = confusion_matrix(y_test, val_knn)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
sns.set(font_scale=2.3) #for label size
sns.heatmap(cm/cm.sum(axis=0), annot=True, annot_kws={"size": 30}, fmt='.2%',
            cbar = False, cmap='Purples', xticklabels=classes, yticklabels=classes);
ax.set_ylabel('True Values',fontsize=30)
ax.set_xlabel('Predicted Values',fontsize=30)
plt.show()

In [ ]:
#@title ***AdaBoost***
ab_estimator = 15

# AdaBoost
ab_clf = ensemble.AdaBoostClassifier(n_estimators=ab_estimator)
ab_clf = ab_clf.fit(x_train, y_train)
val_ab = ab_clf.predict(x_test)

cm = confusion_matrix(y_test, val_ab)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
sns.set(font_scale=2.3) #for label size
sns.heatmap(cm/cm.sum(axis=0), annot=True, annot_kws={"size": 30}, fmt='.2%',
            cbar = False, cmap='Purples', xticklabels=classes, yticklabels=classes);
ax.set_ylabel('True Values',fontsize=30)
ax.set_xlabel('Predicted Values',fontsize=30)
plt.show()

In [ ]:
#@title ***Random Forest***
rf_tree = 5
rf_depth = 10

# Random Forest
rf_clf = ensemble.RandomForestClassifier(n_estimators=rf_tree, max_depth=rf_depth)
rf_clf = rf_clf.fit(x_train, y_train)
val_rf = rf_clf.predict(x_test)

cm = confusion_matrix(y_test, val_rf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
sns.set(font_scale=2.3) #for label size
sns.heatmap(cm/cm.sum(axis=0), annot=True, annot_kws={"size": 30}, fmt='.2%',
            cbar = False, cmap='Purples', xticklabels=classes, yticklabels=classes);
ax.set_ylabel('True Values',fontsize=30)
ax.set_xlabel('Predicted Values',fontsize=30)
plt.show()

In [ ]:
#@title ***SVM***
svm_sup = 1
svm_kernel = 'rbf'

# SVM
svm_clf = svm.SVC(C=svm_sup, kernel=svm_kernel, gamma='auto')
svm_clf = svm_clf.fit(x_train, y_train)
val_svm = svm_clf.predict(x_test)

cm = confusion_matrix(y_test, val_svm)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
sns.set(font_scale=2.3) #for label size
sns.heatmap(cm/cm.sum(axis=0), annot=True, annot_kws={"size": 30}, fmt='.2%',
            cbar = False, cmap='Purples', xticklabels=classes, yticklabels=classes);
ax.set_ylabel('True Values',fontsize=30)
ax.set_xlabel('Predicted Values',fontsize=30)
plt.show()

In [ ]:
#@title ***MLP***
mlp_arch = (20, 10, 10, 20)
mlp_activation = 'tanh'

# MLP
mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_arch), activation=mlp_activation, solver='adam', learning_rate='adaptive', max_iter=500, verbose=False)
mlp_clf = mlp_clf.fit(x_train, y_train)
val_mlp = mlp_clf.predict(x_test)

cm = confusion_matrix(y_test, val_mlp)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
sns.set(font_scale=2.3) #for label size
sns.heatmap(cm/cm.sum(axis=0), annot=True, annot_kws={"size": 30}, fmt='.2%',
            cbar = False, cmap='Purples', xticklabels=classes, yticklabels=classes);
ax.set_ylabel('True Values',fontsize=30)
ax.set_xlabel('Predicted Values',fontsize=30)
plt.show()